In [1]:
import os
import sys
import cv2
import  torch
import  numpy as np
import torch.utils.data as data
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
from torch.autograd import Variable
import torchvision
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import pandas as pd
import random
import pickle
from tqdm import tqdm
import csv

In [2]:
class Imgdataset(data.Dataset):
    def __init__(self, img_folder, ann_data, img_size=64, transform=[]):
        self.img_folder = img_folder
        self.ann_data = ann_data
        self.transform = transform

        self.all_ann_smiling = {}
        with open(ann_data, 'r') as f:
            rows = csv.DictReader(f)
            for row in rows:
                self.all_ann_smiling[row['image_name']] = int(float(row['Smiling']))
        
        self.all_img_path = []
        all_img_file = os.listdir(img_folder)
        for file_name in all_img_file:
            self.all_img_path.append(file_name)

        
    def __len__(self):
        return len(self.all_img_path)   

    def __getitem__(self, index):
        
        img_file_name = self.all_img_path[index]
        label = self.all_ann_smiling[img_file_name]
        
        img_path = os.path.join(self.img_folder, img_file_name)
        img = cv2.imread(img_path)
        img = self.BGR2RGB(img)
        #img = self.random_flip(img)
        for t in self.transform:
            img = t(img)

        return img, label


    def BGR2RGB(self,img):
        return cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

In [15]:
class Generator(nn.Module):
    def __init__(self, figsize=64):
        super(Generator, self).__init__()
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d( 100, figsize * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(figsize * 8),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(figsize * 8, figsize * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(figsize * 4),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(figsize * 4, figsize * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(figsize * 2),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(figsize * 2, figsize, 4, 2, 1, bias=False),
            nn.BatchNorm2d(figsize),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(figsize, 3, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, X):
        output = self.decoder(X)/2.0+0.5
        return output
    
class Discriminator(nn.Module):
    def __init__(self, figsize=64):
        super(Discriminator, self).__init__()
        self.decoder = nn.Sequential(
            nn.Conv2d(3, figsize, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(figsize, figsize * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(figsize * 2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(figsize * 2, figsize * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(figsize * 4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(figsize * 4, figsize * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(figsize * 8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(figsize * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, X):
        output = self.decoder(X)
        #print(output.size())
        return output.squeeze(1)

In [20]:
random.seed((38))
torch.manual_seed(38)
fixed_noise = torch.randn(32, 100, 1, 1)
fixed_noise = Variable(fixed_noise).cuda()

In [21]:
train_file_img = "../../../hw3_data/face/train"
csv_data_path = "../../../hw3_data/face/train.csv"
img_size = 64
train_data = Imgdataset(train_file_img, csv_data_path, img_size, transform=[transforms.ToTensor()])

dataloader = torch.utils.data.DataLoader(
                                dataset=train_data,
                                batch_size=64,
                                shuffle=True,
                                num_workers=8
                            )

In [23]:
# training
latent_size = 100

G = Generator().cuda()
D = Discriminator().cuda()
criterion = nn.BCELoss()

# setup optimizer
optimizerG = optim.Adam(G.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizerD = optim.Adam(D.parameters(), lr=0.0002, betas=(0.5, 0.999))


BATCH_SIZE = 64

D_loss_list = []
G_loss_list = []
D_fake_acc_list = []
D_real_acc_list = []

for epoch in range(105):
    print("Epoch:", epoch+1)
    epoch_D_loss = 0.0
    epoch_G_loss = 0.0
    D_fake_acc = 0.0
    D_real_acc = 0.0
    
    total_length = len(dataloader)
    if (epoch+1) == 11:
        optimizerG.param_groups[0]['lr'] /= 2
        optimizerD.param_groups[0]['lr'] /= 2
        print("learning rate change!")

    if (epoch+1) == 20:
        optimizerG.param_groups[0]['lr'] /= 2
        optimizerD.param_groups[0]['lr'] /= 2
        print("learning rate change!")
        
    for i, batch in enumerate(dataloader):
        if i > total_length:
            break
            
        D.zero_grad()
        input_X = batch[0]
        
        real_image = Variable(input_X.cuda()) # use GPU 
        real_label = Variable(torch.ones((BATCH_SIZE))).cuda()
        output = D(real_image)
        #print(real_label.size())
        #print(output)
        
        D_real_loss = criterion(output, real_label)
        D_real_acc += np.mean(((output > 0.5).cpu().data.numpy() == real_label.cpu().data.numpy()))
        
        #### train with fake image -> ground truth = fake label
        noise = Variable(torch.randn(BATCH_SIZE, 100, 1, 1)).cuda()
        fake_image = G(noise)
        fake_label = Variable(torch.zeros((BATCH_SIZE))).cuda()
        output = D(fake_image.detach())
        D_fake_loss = criterion(output, fake_label)
        D_fake_acc += np.mean(((output > 0.5).cpu().data.numpy() == fake_label.cpu().data.numpy()))
        
        # update D
        D_train_loss = D_real_loss + D_fake_loss
        epoch_D_loss+=(D_train_loss.item())
        D_train_loss.backward()
        optimizerD.step()
        
        #### train Generator
        for _ in range(1):
            G.zero_grad()
            # generate fake image
            noise = Variable(torch.randn(BATCH_SIZE, 100, 1, 1)).cuda()
            fake_image = G(noise)
            fake_label_for_G = Variable(torch.ones((BATCH_SIZE))).cuda()
            output = D(fake_image)
            G_loss = criterion(output, fake_label_for_G)
            epoch_G_loss += (G_loss.item())
            G_loss.backward()
            optimizerG.step()
        
    print("training D Loss:",epoch_D_loss/(total_length))
    print("training G Loss:", epoch_G_loss/(total_length))
    D_loss_list.append(epoch_D_loss/(total_length))
    G_loss_list.append(epoch_G_loss/(total_length))
    
    print("D_real_acc:", D_real_acc/(total_length/BATCH_SIZE))
    print("D_fake_acc:", D_fake_acc/(total_length/BATCH_SIZE))
    
    D_real_acc_list.append(D_real_acc/(total_length/BATCH_SIZE))
    D_fake_acc_list.append(D_fake_acc/(total_length/BATCH_SIZE))
    # evaluation
    G.eval()
    fixed_img_output = G(fixed_noise)
    G.train()
    torchvision.utils.save_image(fixed_img_output.cpu().data, './output_GAN/fig2_3_'+str(epoch+1)+'.jpg',nrow=8)
    torch.save(G.state_dict(), "./output_GAN/G_model.pkt."+str(epoch+1))

Epoch: 1
training D Loss: 0.8201619282126427
training G Loss: 3.1805259925842284
D_real_acc: 51.752
D_fake_acc: 53.6416
Epoch: 2
training D Loss: 0.7773810708999633
training G Loss: 3.1401797554016113
D_real_acc: 52.408
D_fake_acc: 54.0208
Epoch: 3
training D Loss: 0.7060195737838745
training G Loss: 3.0358400404930115
D_real_acc: 54.216
D_fake_acc: 55.1072
Epoch: 4
training D Loss: 0.759030563545227
training G Loss: 2.760032010555267
D_real_acc: 53.08
D_fake_acc: 53.688
Epoch: 5
training D Loss: 0.7813392647743225
training G Loss: 2.580987536239624
D_real_acc: 52.8352
D_fake_acc: 53.7088
Epoch: 6
training D Loss: 0.7429836676597595
training G Loss: 2.5491879749298096
D_real_acc: 53.6224
D_fake_acc: 54.0208
Epoch: 7
training D Loss: 0.7133242322444916
training G Loss: 2.494649712228775
D_real_acc: 54.1248
D_fake_acc: 54.6544
Epoch: 8
training D Loss: 0.6856780827045441
training G Loss: 2.550289152240753
D_real_acc: 54.7696
D_fake_acc: 55.2592
Epoch: 9
training D Loss: 0.654060267066955

training D Loss: 0.07445280629992485
training G Loss: 4.814301338958741
D_real_acc: 63.8816
D_fake_acc: 63.7296
Epoch: 70
training D Loss: 0.1405616348683834
training G Loss: 4.758867382198572
D_real_acc: 63.1744
D_fake_acc: 63.0288
Epoch: 71
training D Loss: 0.13508083447664976
training G Loss: 4.7202636922836305
D_real_acc: 63.2432
D_fake_acc: 63.0416
Epoch: 72
training D Loss: 0.07246173892766238
training G Loss: 4.875321855926513
D_real_acc: 63.8208
D_fake_acc: 63.7056
Epoch: 73
training D Loss: 0.15760874087512494
training G Loss: 4.665402967396378
D_real_acc: 62.9856
D_fake_acc: 62.8336
Epoch: 74
training D Loss: 0.12501255050748586
training G Loss: 4.841278453826904
D_real_acc: 63.136
D_fake_acc: 63.0448
Epoch: 75
training D Loss: 0.09864634716808796
training G Loss: 4.651648036336899
D_real_acc: 63.4944
D_fake_acc: 63.4208
Epoch: 76
training D Loss: 0.089897746104002
training G Loss: 4.896641481590271
D_real_acc: 63.6128
D_fake_acc: 63.4192
Epoch: 77
training D Loss: 0.12838010

In [36]:
### Load Model ###
G = Generator()
G.cuda()
pretrained_acgan = "./output_GAN/G_model.pkt.105"
G.load_state_dict(torch.load(pretrained_acgan))
G.eval()

random_seed = [72, 76, 88, 110, 159, 191, 198, 213,
               229, 249, 303, 316, 338, 339, 348, 355, 
               363, 393, 414, 427, 430, 441, 447, 448, 
               547, 553, 555, 583, 632, 633, 673, 702]
output = []
for seed in random_seed:
    
    random.seed((seed))
    torch.manual_seed(seed)
    fixed_noise = torch.randn(1, 100, 1, 1)
    fixed_noise = Variable(fixed_noise).cuda()
    
    fixed_img_output = G(fixed_noise)
    output.append(fixed_img_output.cpu().data)
output = torch.cat(output, 0)
torchvision.utils.save_image(output, './GAN_seed/seed_'+"ALL"+'.jpg',nrow=8)
